In [ ]:
import pandas as pd

In [82]:
def find_phase_index_ratings(df, edo):
    working_dict = {'components' : [],
    'rating' : []}
    for x in range(1, edo//2 +1):
        for y in range(x+1, edo//2 + 1):
            for z in range(1, edo//2 + 1):
                if z != x and z != y:
                    working_dict['components'].append(str([x, y, z]))
                    phase_index_list = (abs(df[f'f{x} Phase'] + df[f'f{y} Phase'] - df[f'f{z} Phase'])).values.tolist()
                    norm = [fix_index(a) for a in phase_index_list]
                    working_dict['rating'].append(sum(norm) / len(norm))
    return working_dict

In [83]:
haba = pd.read_csv("C:\\Users\\jennd\\Desktop\\haba_master_df.csv")

In [84]:
haba.head()

,Unnamed: 0,Window Number,Weighted Array,Original Array,Measure Range,f1 Phase,f2 Phase,f3 Phase,f4 Phase,f5 Phase,...,f3 Magnitude,f4 Magnitude,f5 Magnitude,f6 Magnitude,f7 Magnitude,f8 Magnitude,f9 Magnitude,f10 Magnitude,f11 Magnitude,f12 Magnitude
0,0,0,[1.58 1. 1.58 2. 0. 1. 1. 1. 3. ...,[2. 1. 2. 3. 0. 1. 1. 1. 7. 1. 0. 2. 0. 1. 0. ...,1–5,-88,-83,-9,-117,-99,...,6.25,2.92,0.33,7.01,4.78,1.88,7.00,2.73,1.72,4.58
1,1,1,[1.58 1. 1.58 2. 1. 1. 1. 1. 3. ...,[2. 1. 2. 3. 1. 1. 1. 1. 7. 1. 0. 2. 0. 1. 0. ...,1–5,-74,-87,-11,-131,78,...,5.27,1.83,1.26,7.62,4.07,1.83,6.00,3.38,2.30,3.58
2,2,2,[2. 1. 1.58 2. 1. 0. 1. 1. 3.17 ...,[3. 1. 2. 3. 1. 0. 1. 1. 8. 1. 0. 3. 0. 1. 0. ...,1–5,-76,-68,-21,-120,134,...,7.39,1.34,2.08,8.32,1.98,2.17,5.67,3.41,4.57,3.42
3,3,3,[2.32 1. 1. 2. 1. 0. 1. 1. 3.17 ...,[4. 1. 1. 3. 1. 0. 1. 1. 8. 1. 0. 3. 0. 1. 0. ...,1–5,-84,-67,-14,-126,102,...,7.71,0.81,1.53,9.25,1.19,1.37,5.79,3.39,4.62,4.09
4,4,4,[2.32 1. 1.58 2. 1. 0. 1. 1. 3.17 ...,[4. 1. 2. 3. 1. 0. 1. 1. 8. 1. 1. 3. 0. 1. 0. ...,2–6,-88,-33,-26,-148,77,...,7.86,1.24,1.09,7.97,1.23,1.51,5.08,2.65,5.64,2.92


In [85]:
haba_df = pd.DataFrame(find_phase_index_ratings(haba, 24))
haba_df

,components,rating
0,"[1, 2, 3]",90.979310
1,"[1, 2, 4]",95.337931
2,"[1, 2, 5]",91.452217
3,"[1, 2, 6]",87.969458
4,"[1, 2, 7]",92.494581
...,...,...
655,"[11, 12, 6]",86.450246
656,"[11, 12, 7]",95.503448
657,"[11, 12, 8]",92.512315
658,"[11, 12, 9]",86.511330


In [86]:
min(haba_df['rating'])

77.84630541871921

In [87]:
haba_df.loc[haba_df['rating'] == min(haba_df['rating'])]

,components,rating
312,"[4, 6, 3]",77.846305


In [88]:
haba_df.loc[haba_df['rating'] < 80]

,components,rating
18,"[1, 3, 11]",79.579310
218,"[3, 4, 11]",79.578325
312,"[4, 6, 3]",77.846305
404,"[5, 8, 6]",79.389163
410,"[5, 9, 1]",79.983251
520,"[7, 9, 1]",79.506404
